In [0]:
spark.sql("Use Catalog incremental_data_loading_catalog")

staging_df = spark.read.table('order_details')

staging_df.show()

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit
transformed_df = staging_df.withColumn("LastUpdated", current_timestamp()).withColumn('timeZone', lit('America/Los_Angeles'))

transformed_df.show()

In [0]:
#spark.sql('Drop table if exists order_details_final')
spark.sql("Create table if not exists order_details_final (OrderID int, CustomerID string, OrderDate string, ProductID string, Quantity int, UnitPrice double, LastUpdated timestamp, timeZone string)")

In [0]:
from delta.tables import DeltaTable

target_df = DeltaTable.forName(spark, 'order_details_final')

In [0]:
from delta.tables import DeltaTable

target_df.alias('final').merge(transformed_df.alias('stage'), 'final.OrderID = stage.OrderID') \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

print('Target loaded successfully')